In [1]:
import simpy
import random
import math
import numpy

In [2]:
RANDOM_SEED = 9782

INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [3]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Duration of waiting time of the customer between automatic operator and op1 or op2
queue_waiting = []



# The number of customer in the answering system
queue_w_customer = []

# The number of customer in the answering system
queue_w_operator1 = []

# The number of customer in the answering system
queue_w_operator2 = []




# Simulation end time
end_time = 0;



In [4]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.queue_t=0
    def call(self):
        global end_time 
        
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        
        prob=random.uniform(0,10)
        print('prob',prob)
        error=random.uniform(0,10)
        print('error',error)
        print('otomatik call len',len(queue_w_customer)) 
        
        
     
        if len(queue_w_customer)<101:
            yield env.timeout(random.expovariate(CALL_RATE))
            print('%s otomatik operatorden cikti at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            self.queue_t=self.env.now
            if error>1 :
                if prob<3:
                    queue_w_operator1.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call1())

                if prob>=3:  
                    queue_w_operator2.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call2())

                
            else:
                self.end_time = self.env.now
                print('%s A caller hangs up -error- at %g' % (self.name, self.env.now))
        else:
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            print('drop len',len(queue_w_customer)) 


            
    def call1(self):
        
        with operator1.request() as req:
            yield req

            print('bekleme',self.env.now - self.queue_t)
            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
                
                yield self.env.process(self.get_help1_operator1())
                print('%s is done operator1 at %g' % (self.name, self.env.now))
                self.end_time = self.env.now
            else:
                queue_waiting.append(10)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                
                self.end_time = self.queue_t+10
            queue_w_operator1.pop()


    def call2(self):
        
        with operator2.request() as req:
            yield req

            print('bekleme',self.env.now - self.queue_t)
            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
                yield self.env.process(self.get_help2_operator2())
                print('%s is done operator2 at %g' % (self.name, self.env.now))
                self.end_time = self.env.now
            else:
                queue_waiting.append(10)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                self.end_time = self.queue_t+10
            queue_w_operator2.pop()

      
            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [5]:

def customer_generator(env, operator1, operator2):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [6]:
def break_generator1(env, operator1):
    """Generate new break for operator1."""
    BREAK_TIME_COUNTER1 = 0
    
    while True:
      
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting1', len(queue_w_operator1))
        while True:
            with operator1.request() as req:
                yield req
                
                if queue_w_operator1:
                    continue
                else:
                    print('Operator1 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator1 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER1+=1
                    print('break1',BREAK_TIME_COUNTER1)
                    break

      

In [7]:
def break_generator2(env, operator2):
    """Generate new break for operator2."""
    
    BREAK_TIME_COUNTER2 = 0
    
    while True:
        
           
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting2', len(queue_w_operator2))

        while True:
            with operator2.request() as req:
                yield req
                if queue_w_operator2:
                    continue
                else:
                    print('Operator2 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator2 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER2+=1
                    print('break2',BREAK_TIME_COUNTER2)
                    break

In [8]:
#variables of statistics

#sum of 10 1000 customer data utilization of operator1 
sumUtil1KOP1=0
#sum of 10 1000 customer data utilization of operator2 
sumUtil1KOP2=0

#sum of waiting times of 1k customer
sumOf1kWaitingTimes=0;






#sum of 10 5000 customer data utilization of operator1 
sumUtil5KOP1=0
#sum of 10 5000 customer data utilization of operator2 
sumUtil5KOP2=0

#sum of waiting times of 5k customer
sumOf5kWaitingTimes=0;


In [9]:
#1k
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=7000)
    
    #statistic calculations
    
    
    print(end_time)
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil1KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil1KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf1kWaitingTimes+=sum(queue_waiting)
    
    
    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []
    
    queue_waiting = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0;
    
    

Customer 1 initiated a call at 3.01324
prob 3.329752310576437
error 1.469620373250936
otomatik call len 1
Customer 2 initiated a call at 3.5674
prob 8.903580632740999
error 8.846221745223051
otomatik call len 2
Customer 3 initiated a call at 3.92895
prob 4.020126375366169
error 0.20366493269210584
otomatik call len 3
Customer 4 initiated a call at 3.98808
prob 9.070409264074497
error 8.646734989969438
otomatik call len 4
Customer 2 otomatik operatorden cikti at 4.98243
bekleme 0.0
Customer 2 is assigned to a operator2 at 4.98243
Customer 1 otomatik operatorden cikti at 5.82933
Customer 3 otomatik operatorden cikti at 7.64086
Customer 3 A caller hangs up -error- at 7.64086
Customer 5 initiated a call at 7.71364
prob 5.836776401092649
error 7.75640198599217
otomatik call len 2
Customer 2 is done operator2 at 9.63242
bekleme 3.803091037038154
Customer 1 is assigned to a operator2 at 9.63242
Customer 4 otomatik operatorden cikti at 12.0826
Customer 1 is done operator2 at 14.4318
bekleme 2.

Customer 143 is assigned to a operator2 at 873.989
Customer 147 initiated a call at 874.723
prob 5.046520910759512
error 0.2567444307092881
otomatik call len 1
Customer 147 otomatik operatorden cikti at 874.925
Customer 147 A caller hangs up -error- at 874.925
Operator1 break done at 876.274
break1 7
Customer 143 is done operator2 at 878.402
bekleme 8.894113925317356
Customer 140 is assigned to a operator2 at 878.402
Customer 140 is done operator2 at 880.532
bekleme 9.192097149318784
Customer 146 is assigned to a operator2 at 880.532
Customer 146 is done operator2 at 884.325
Customer 148 initiated a call at 889.15
prob 9.360642459509377
error 7.4893875821038485
otomatik call len 1
Customer 149 initiated a call at 889.63
prob 2.5792889165340926
error 2.6257256559459776
otomatik call len 2
Customer 150 initiated a call at 889.772
prob 0.7048718130905862
error 9.465333424626378
otomatik call len 3
Customer 148 otomatik operatorden cikti at 891.077
bekleme 0.0
Customer 148 is assigned to a

error 9.046514312034999
otomatik call len 3
Customer 267 otomatik operatorden cikti at 1711.37
bekleme 0.0
Customer 267 is assigned to a operator1 at 1711.37
Customer 263 is done operator2 at 1711.77
bekleme 0.9050691699947038
Customer 266 is assigned to a operator2 at 1711.77
Customer 264 otomatik operatorden cikti at 1711.95
Customer 268 otomatik operatorden cikti at 1715.22
Customer 269 initiated a call at 1715.26
prob 6.668794537878716
error 1.3066744198761637
otomatik call len 1
Customer 269 otomatik operatorden cikti at 1716.71
Customer 266 is done operator2 at 1717.5
bekleme 6.4453446021923355
Customer 265 is assigned to a operator2 at 1717.5
Customer 265 is done operator2 at 1718.4
bekleme 3.1862910960212503
Customer 268 is assigned to a operator2 at 1718.4
Customer 270 initiated a call at 1718.97
prob 2.1741625382120944
error 9.399078973949127
otomatik call len 1
Customer 268 is done operator2 at 1719.3
bekleme 2.596577989360185
Customer 269 is assigned to a operator2 at 1719.

prob 7.705380541868431
error 4.447437926619548
otomatik call len 1
Customer 391 initiated a call at 2391.1
prob 0.39793111065178133
error 4.565759916522982
otomatik call len 2
Customer 391 otomatik operatorden cikti at 2392.94
Customer 389 is done operator2 at 2394.87
Customer 392 initiated a call at 2395.33
prob 1.8502910692354069
error 7.333226054623447
otomatik call len 2
Customer 384 is done operator1 at 2395.61
bekleme 2.676108136181938
Customer 391 is assigned to a operator1 at 2395.61
Customer 392 otomatik operatorden cikti at 2396.59
Customer 390 otomatik operatorden cikti at 2397.78
bekleme 0.0
Customer 390 is assigned to a operator2 at 2397.78
Customer 393 initiated a call at 2401.08
prob 8.418702226526047
error 2.134239614818352
otomatik call len 1
Customer 394 initiated a call at 2401.16
prob 4.534070664957563
error 6.002060880649848
otomatik call len 2
Customer 394 otomatik operatorden cikti at 2402.92
Customer 390 is done operator2 at 2403.46
bekleme 0.542781514908711
Cus

waiting2 2
Customer 513 otomatik operatorden cikti at 3058.37
bekleme 0.0
Customer 513 is assigned to a operator1 at 3058.37
Customer 510 is done operator2 at 3059.15
bekleme 10.776730972314454
Customer 508 A caller hangs up-10 min- at 3058.37
Operator2 gives break at 3059.15
Customer 511 otomatik operatorden cikti at 3062.01
Operator2 break done at 3062.15
break2 44
bekleme 0.13963366482494166
Customer 511 is assigned to a operator2 at 3062.15
Customer 511 is done operator2 at 3065.35
Customer 512 otomatik operatorden cikti at 3065.72
bekleme 0.0
Customer 512 is assigned to a operator2 at 3065.72
Customer 514 initiated a call at 3067.15
prob 4.90588180329604
error 3.533361034335446
otomatik call len 1
waiting1 1
Customer 512 is done operator2 at 3071.78
Customer 515 initiated a call at 3077.08
prob 8.633293516421608
error 6.237626074142451
otomatik call len 2
Customer 514 otomatik operatorden cikti at 3077.78
bekleme 0.0
Customer 514 is assigned to a operator2 at 3077.78
Customer 515 

break2 57
bekleme 2.7671469745209833
Customer 639 is assigned to a operator2 at 3835.53
Customer 639 is done operator2 at 3838.13
waiting2 0
Operator2 gives break at 3839.39
Customer 640 is done operator1 at 3841.2
bekleme 11.818644229891106
Customer 638 A caller hangs up-10 min- at 3839.38
bekleme 10.545346433342274
Customer 642 A caller hangs up-10 min- at 3840.65
Operator2 break done at 3842.39
break2 58
Customer 643 initiated a call at 3850.47
prob 0.07884576014911326
error 3.5838090804114318
otomatik call len 1
Customer 644 initiated a call at 3852.19
prob 0.4206191564188244
error 8.65117337874887
otomatik call len 2
Customer 645 initiated a call at 3853.2
prob 2.436434472625091
error 5.587881096431234
otomatik call len 3
Customer 646 initiated a call at 3859.59
prob 3.833366777767896
error 0.6978970066760337
otomatik call len 4
Customer 643 otomatik operatorden cikti at 3860.14
bekleme 0.0
Customer 643 is assigned to a operator1 at 3860.14
Customer 646 otomatik operatorden cikti 

bekleme 0.0
Customer 763 is assigned to a operator2 at 4576.12
Customer 763 is done operator2 at 4580.8
Customer 764 initiated a call at 4584.31
prob 5.15478579425055
error 3.517368596464805
otomatik call len 1
Customer 765 initiated a call at 4584.4
prob 6.420587141410092
error 8.17097961747992
otomatik call len 2
Customer 765 otomatik operatorden cikti at 4585.2
bekleme 0.0
Customer 765 is assigned to a operator2 at 4585.2
waiting2 1
Customer 764 otomatik operatorden cikti at 4587.36
Customer 766 initiated a call at 4588.45
prob 2.865892800175287
error 0.3088219422489735
otomatik call len 1
Customer 767 initiated a call at 4589.63
prob 1.4475214778986112
error 0.6596661130763826
otomatik call len 2
waiting1 0
Operator1 gives break at 4590.95
Customer 765 is done operator2 at 4591.18
bekleme 3.8258343093839358
Customer 764 is assigned to a operator2 at 4591.18
Customer 764 is done operator2 at 4593.41
Operator2 gives break at 4593.41
Customer 767 otomatik operatorden cikti at 4593.8
C

otomatik call len 1
Customer 879 is done operator2 at 5292.15
Customer 880 is done operator1 at 5293.55
Customer 882 initiated a call at 5294.71
prob 7.143202830865897
error 0.1420274110903741
otomatik call len 2
Customer 883 initiated a call at 5294.81
prob 8.168607088001487
error 7.563289431881133
otomatik call len 3
Customer 881 otomatik operatorden cikti at 5296.29
bekleme 0.0
Customer 881 is assigned to a operator1 at 5296.29
Customer 883 otomatik operatorden cikti at 5296.84
bekleme 0.0
Customer 883 is assigned to a operator2 at 5296.84
waiting1 1
Customer 884 initiated a call at 5300.72
prob 9.586801054008289
error 8.261813713181766
otomatik call len 2
Customer 881 is done operator1 at 5301.73
Operator1 gives break at 5301.73
Customer 883 is done operator2 at 5301.92
Customer 884 otomatik operatorden cikti at 5302.32
bekleme 0.0
Customer 884 is assigned to a operator2 at 5302.32
Operator1 break done at 5304.73
break1 70
Customer 884 is done operator2 at 5307.68
Customer 882 otom

error 1.9364386831630243
otomatik call len 1
Customer 990 otomatik operatorden cikti at 6004.94
Customer 989 is done operator2 at 6005.59
bekleme 0.6490310751305515
Customer 990 is assigned to a operator2 at 6005.59
Customer 991 initiated a call at 6007.97
prob 3.1017585582095064
error 9.763996962567848
otomatik call len 1
Customer 991 otomatik operatorden cikti at 6008.1
waiting2 2
Customer 990 is done operator2 at 6010.85
bekleme 2.7533188785491802
Customer 991 is assigned to a operator2 at 6010.85
Customer 992 initiated a call at 6012.38
prob 8.079055395485206
error 7.929668198835089
otomatik call len 1
Customer 992 otomatik operatorden cikti at 6013.21
Customer 991 is done operator2 at 6017.73
bekleme 4.51654586064069
Customer 992 is assigned to a operator2 at 6017.73
Customer 992 is done operator2 at 6022.18
Operator2 gives break at 6022.18
Customer 993 initiated a call at 6022.31
prob 9.553528696598105
error 2.1908970608774037
otomatik call len 1
Customer 993 otomatik operatorden

ZeroDivisionError: float division by zero

In [ ]:
#1k

#utilization average of operator 1
utilOf1KOP1=sumUtil1KOP1/10

#utilization average of operator 2
utilOf1KOP2=sumUtil1KOP2/10

#average of waiting times 
avg1kWaitingTimes=sumOf1kWaitingTimes/10


print('1K AVERAGE STATISTICS')


print('Utilization of operator1 %g' % utilOf1KOP1)
print('Utilization of operator2 %g' % utilOf1KOP2)
print('Average waiting times of customers %g' % avg1kWaitingTimes)



In [ ]:
CUSTOMER_COUNT=5000
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=32000)
    
     #statistic calculations
    
    
    
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil5KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil5KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf5kWaitingTimes+=sum(queue_waiting)
    
    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []

    queue_waiting = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0;

In [ ]:
#5k

#utilization average of operator 1
utilOf5KOP1=sumUtil5KOP1/10

#utilization average of operator 2
utilOf5KOP2=sumUtil5KOP2/10

#average of waiting times 
avg5kWaitingTimes=sumOf5kWaitingTimes/10